In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

     |████████████████████████████████| 1.5MB 11.5MB/s 
     |████████████████████████████████| 2.9MB 48.3MB/s 
     |████████████████████████████████| 890kB 48.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=44b4a77f4f792a4c7ff84270b9145a652f09950561c7aa2eb5d80c253586ddae
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
# !pip install lineflow

In [3]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import re
# import lineflow as lf
import IPython
import graphviz
import datetime
import os
import ast
import torch

from sklearn.metrics import accuracy_score
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, LearningRateScheduler
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras import Model

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

INFO:tensorflow:Initializing the TPU system: grpc://10.12.212.82:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.12.212.82:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU')]


# RACE dataset.

In [ ]:
import json

In [ ]:
label_map = {'A':0,
             'B':1,
             'C':2,
             'D':3,
             'E':4}

In [ ]:
def create_dataset(root_dir):
    '''
    Given a file_path, this function will return dataframe with columns article,
    question, options and answer.
    '''
    list_of_questions = []
    list_of_articles = []
    list_of_answers = []
    list_of_options = []

    for i in ('middle','high'):

        dir_path = os.path.join(root_dir,i)
        files = os.listdir(dir_path)

        for i in files:

            f = open(os.path.join(dir_path,i),'r')
            contents = f.read()
            js = json.loads(contents)

            questions = js['questions']
            article = js['article']
            answers = js['answers']
            options = js['options']

            for j in range(len(answers)):
                question = questions[j]
                answer = answers[j]
                option = options[j]

                list_of_answers.append(answer)
                list_of_questions.append(question)
                list_of_articles.append(article)
                list_of_options.append(option)
            
            f.close()
    
    data = pd.DataFrame()
    data['answerKey'] = list_of_answers
    data['article'] = list_of_articles
    data['question'] = list_of_questions
    data['options'] = list_of_options

    return data

In [ ]:
# train_data = create_dataset('/content/drive/MyDrive/RACE/train')
# dev_data = create_dataset('/content/drive/MyDrive/RACE/dev')
# test_data = create_dataset('/content/drive/MyDrive/RACE/test')

In [ ]:
# train_data.to_csv('/content/drive/MyDrive/race_train_data.csv', index=False)
# dev_data.to_csv('/content/drive/MyDrive/race_dev_data.csv', index=False)
# test_data.to_csv('/content/drive/MyDrive/race_test_data.csv', index=False)

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/RACE files/race_train_data.csv')
dev_data = pd.read_csv('/content/drive/MyDrive/RACE files/race_dev_data.csv')
test_data = pd.read_csv('/content/drive/MyDrive/RACE files/race_test_data.csv')

In [ ]:
train_data.head()

,answerKey,article,question,options
0,B,Do you like painting? Have you ever seen a pai...,What special things can you see on a road in S...,"['Some colourful trees.', 'Paintings in tree h..."
1,A,Do you like painting? Have you ever seen a pai...,Wang Yue paints the tree hole paintings to .,"['make the ugly tree holes beautiful', 'make a..."
2,C,Do you like painting? Have you ever seen a pai...,What does Wang Yue worry about before she begi...,"['The cost she will spend.', 'The thoughts fro..."
3,B,Do you like painting? Have you ever seen a pai...,What does Wang Yue think of painting on a tree...,"[""It's a boring job."", ""It's a hard job."", ""It..."
4,D,Do you like painting? Have you ever seen a pai...,What's the best title for this passage?,"['Trees in Shijiazhuang', 'Making Our Roads Be..."


## Fine tuning BERT with RACE.

### Preparing Data.

In [ ]:
def race_preprocessor(dataset, tokenizer):
    '''
    This function will convert a given article, question, choices in a format:

    [CLS] article [SEP] question choices[0] [SEP]
    [CLS] article [SEP] question choices[1] [SEP]
    [CLS] article [SEP] question choices[2] [SEP]
    [CLS] article [SEP] question choices[3] [SEP]

    After converting in this format the data will be tokenized using a given tokenizer.
    This function will return 4 arrays namely, input_ids, attention_mask, token_type_ids and labels.

    individual input_ids, token_type_ids, attention_mask shape will be as: [num_choices, max_seq_length]
    '''
    all_input_ids = []
    all_attention_mask = []
    all_token_type_ids = []

    for i in range(len(dataset)):

        article = dataset['article'].iloc[i]
        question = dataset['question'].iloc[i]
        options = dataset['options'].iloc[i]
        choice_features = []

        for j in range(len(options)):

            option = options[j]
            ques_opt = question + ' ' + f'{j}' + ' ' + option
            ques_opt = re.sub(r'\s+', ' ', ques_opt)

            inputs = tokenizer(article,
                               ques_opt,
                               max_length=MAX_LEN,
                               truncation='only_first',
                               add_special_tokens=True,
                               padding='max_length')
            
            input_ids = inputs['input_ids']
            attention_mask = inputs['attention_mask']
            token_type_ids = inputs['token_type_ids']

            assert len(input_ids) == MAX_LEN
            assert len(token_type_ids) == MAX_LEN
            assert len(attention_mask) == MAX_LEN

            choice_features.append({'input_ids':input_ids,
                                    'attention_mask':attention_mask,
                                    'token_type_ids':token_type_ids})

        all_input_ids.append(np.asarray([cf['input_ids'] for cf in choice_features], dtype='int32'))
        all_attention_mask.append(np.asarray([cf['attention_mask'] for cf in choice_features], dtype='int32'))
        all_token_type_ids.append(np.asarray([cf['token_type_ids'] for cf in choice_features], dtype='int32'))

    return all_input_ids, all_attention_mask, all_token_type_ids

In [ ]:
from transformers import BertTokenizer, TFBertForMultipleChoice

In [ ]:
MAX_LEN = 512
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
dev_input_ids, dev_attention_mask, dev_token_type_ids = race_preprocessor(dataset=dev_data, tokenizer=tokenizer)

In [ ]:
train_input_ids, train_attention_mask, train_token_type_ids = race_preprocessor(dataset=train_data, tokenizer=tokenizer)

In [ ]:
test_input_ids, test_attention_mask, test_token_type_ids = race_preprocessor(dataset=test_data, tokenizer=tokenizer)

In [ ]:
train_input_ids = np.asarray(train_input_ids, dtype='int32')
train_token_type_ids = np.asarray(train_token_type_ids, dtype='int32')
train_attention_mask = np.asarray(train_attention_mask, dtype='int32')

In [ ]:
dev_input_ids = np.asarray(dev_input_ids, dtype='int32')
dev_token_type_ids = np.asarray(dev_token_type_ids, dtype='int32')
dev_attention_mask = np.asarray(dev_attention_mask, dtype='int32')

In [ ]:
test_input_ids = np.asarray(test_input_ids, dtype='int32')
test_token_type_ids = np.asarray(test_token_type_ids, dtype='int32')
test_attention_mask = np.asarray(test_attention_mask, dtype='int32')

In [ ]:
np.save('/content/drive/MyDrive/race_train_input_ids', train_input_ids)
np.save('/content/drive/MyDrive/race_train_token_type_ids', train_token_type_ids)
np.save('/content/drive/MyDrive/race_train_attention_mask', train_attention_mask)

In [ ]:
np.save('/content/drive/MyDrive/race_dev_input_ids', dev_input_ids)
np.save('/content/drive/MyDrive/race_dev_token_type_ids', dev_token_type_ids)
np.save('/content/drive/MyDrive/race_dev_attention_mask', dev_attention_mask)

In [ ]:
np.save('/content/drive/MyDrive/race_test_input_ids', test_input_ids)
np.save('/content/drive/MyDrive/race_test_token_type_ids', test_token_type_ids)
np.save('/content/drive/MyDrive/race_test_attention_mask', test_attention_mask)

### Training Data.

In [ ]:
train_input_ids = np.load('/content/drive/MyDrive/RACE files/BERT files/race_train_input_ids.npy')
train_token_type_ids = np.load('/content/drive/MyDrive/RACE files/BERT files/race_train_token_type_ids.npy')
train_attention_mask = np.load('/content/drive/MyDrive/RACE files/BERT files/race_train_attention_mask.npy')

In [ ]:
train_labels = train_data['answerKey'].map(label_map)

In [ ]:
train_labels.values

array([1, 0, 2, ..., 2, 2, 3])

In [ ]:
train_dict = {'input_ids':train_input_ids,
              'attention_mask':train_attention_mask,
              'token_type_ids':train_token_type_ids}

In [ ]:
viola = tf.data.Dataset.from_tensor_slices((train_dict,train_labels.values))

In [ ]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [ ]:
viola = viola.shuffle(10).batch(8).cache().prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
from transformers import TFBertForMultipleChoice

with strategy.scope():
    model = TFBertForMultipleChoice.from_pretrained('bert-base-uncased')
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(optimizer=optimizer, 
                  loss=loss, metrics=['accuracy'])

In [ ]:
viola

<PrefetchDataset shapes: ({input_ids: (None, 4, 512), attention_mask: (None, 4, 512), token_type_ids: (None, 4, 512)}, (None,)), types: ({input_ids: tf.int32, attention_mask: tf.int32, token_type_ids: tf.int32}, tf.int64)>

In [ ]:
model.fit(viola, epochs=4)

Epoch 1/4
10988/10988 [==============================] - 2191s 199ms/step - loss: 1.0943 - accuracy: 0.5268
Epoch 2/4
10988/10988 [==============================] - 2189s 199ms/step - loss: 0.7879 - accuracy: 0.6838
Epoch 3/4
10988/10988 [==============================] - 2195s 200ms/step - loss: 0.5474 - accuracy: 0.7865
Epoch 4/4
10988/10988 [==============================] - 2196s 200ms/step - loss: 0.3699 - accuracy: 0.8578


In [ ]:
model.save_pretrained('./')

## Fine tuning Roberta with RACE.

In [ ]:
from transformers import RobertaTokenizer, TFRobertaForMultipleChoice

In [ ]:
MAX_LEN = 450
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)

In [ ]:
def race_preprocessor(dataset, tokenizer):
    '''
    This function will convert a given article, question, choices in a format:

    <s> article </s> question </s> choices[0] </s>
    <s> article </s> question </s> choices[1] </s>
    <s> article </s> question </s> choices[2] </s>
    <s> article </s> question </s> choices[3] </s>

    After converting in this format the data will be tokenized using a given tokenizer.
    This function will return 4 arrays namely, input_ids, attention_mask, token_type_ids and labels.

    individual input_ids, token_type_ids, attention_mask shape will be as: [num_choices, max_seq_length]
    '''
    all_input_ids = []
    all_attention_mask = []
    all_token_type_ids = []

    for i in range(len(dataset)):

        article = dataset['article'].iloc[i]
        question = dataset['question'].iloc[i]
        options = ast.literal_eval(dataset['options'].iloc[i])
        choice_features = []

        for j in range(len(options)):

            option = options[j]
            input_string = '<s>' + ' ' + article + ' ' + '</s>' + ' ' + question + ' ' + '</s>' + ' ' + option + ' ' + '</s>'
            input_string = re.sub(r'\s+', ' ', input_string)

            input_ids = tokenizer(input_string,
                               max_length=MAX_LEN,
                               add_special_tokens=False)['input_ids']
            
            attention_mask = [1] * len(input_ids)

            padding_id = tokenizer.pad_token_id
            padding_length = 450 - len(input_ids)

            input_ids = input_ids + [padding_id]*padding_length
            attention_mask = attention_mask + [0]*padding_length

            assert len(input_ids) == MAX_LEN
            assert len(attention_mask) == MAX_LEN

            choice_features.append({'input_ids':input_ids,
                                    'attention_mask':attention_mask})

        all_input_ids.append(np.asarray([cf['input_ids'] for cf in choice_features], dtype='int32'))
        all_attention_mask.append(np.asarray([cf['attention_mask'] for cf in choice_features], dtype='int32'))

    return all_input_ids, all_attention_mask

In [ ]:
dev_input_ids, dev_attention_mask = race_preprocessor(dataset=dev_data, tokenizer=tokenizer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
dev_input_ids[0]

array([[   0, 1308, 1441, ...,    1,    1,    1],
       [   0, 1308, 1441, ...,    1,    1,    1],
       [   0, 1308, 1441, ...,    1,    1,    1],
       [   0, 1308, 1441, ...,    1,    1,    1]], dtype=int32)

In [ ]:
dev_input_ids = np.asarray(dev_input_ids, dtype='int32')
dev_attention_mask = np.asarray(dev_attention_mask, dtype='int32')

In [ ]:
np.save('/content/drive/MyDrive/RACE files/RoBERTa files/race_dev_input_ids', dev_input_ids)
np.save('/content/drive/MyDrive/RACE files/RoBERTa files/race_dev_attention_mask', dev_attention_mask)

In [ ]:
test_input_ids, test_attention_mask = race_preprocessor(dataset=test_data, tokenizer=tokenizer)

In [ ]:
test_input_ids = np.asarray(test_input_ids, dtype='int32')
test_attention_mask = np.asarray(test_attention_mask, dtype='int32')

In [ ]:
np.save('/content/drive/MyDrive/RACE files/RoBERTa files/race_test_input_ids', test_input_ids)
np.save('/content/drive/MyDrive/RACE files/RoBERTa files/race_test_attention_mask', test_attention_mask)

In [ ]:
train_input_ids, train_attention_mask = race_preprocessor(dataset=train_data, tokenizer=tokenizer)

In [ ]:
train_input_ids = np.asarray(train_input_ids, dtype='int32')
train_attention_mask = np.asarray(train_attention_mask, dtype='int32')

In [ ]:
len(train_input_ids)

87897

In [ ]:
train_attention_mask[0]

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)

In [ ]:
np.save('/content/drive/MyDrive/RACE files/RoBERTa files/race_train_input_ids', train_input_ids)
np.save('/content/drive/MyDrive/RACE files/RoBERTa files/race_train_attention_mask', train_attention_mask)

#### Training

In [ ]:
train_input_ids = np.load('/content/drive/MyDrive/RACE files/RoBERTa files/race_train_input_ids.npy')
train_attention_mask = np.load('/content/drive/MyDrive/RACE files/RoBERTa files/race_train_attention_mask.npy')

In [ ]:
train_labels = train_data['answerKey'].map(label_map)

In [ ]:
train_labels.values

array([1, 0, 2, ..., 2, 2, 3])

In [ ]:
train_dict = {'input_ids':train_input_ids,
              'attention_mask':train_attention_mask}

In [ ]:
viola = tf.data.Dataset.from_tensor_slices((train_dict,train_labels.values))

In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
viola = viola.shuffle(32).batch(16).cache().prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
with strategy.scope():

    model = TFRobertaForMultipleChoice.from_pretrained('roberta-base')

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    model.compile(optimizer=optimizer, 
                  loss=loss, metrics=['accuracy'])

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaForMultipleChoice: ['lm_head']
- This IS expected if you are initializing TFRobertaForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFRobertaForMultipleChoice were not initialized from the model checkpoint at roberta-base and are newly initialized: ['dropout_113', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
viola

<PrefetchDataset shapes: ({input_ids: (None, 4, 450), attention_mask: (None, 4, 450)}, (None,)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.int64)>

In [ ]:
model.fit(viola, epochs=4)

Epoch 1/4
   2/5494 [..............................] - ETA: 28:35:05 - loss: 1.3967 - accuracy: 0.2500WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0089s vs `on_train_batch_end` time: 0.2702s). Check your callbacks.


5494/5494 [==============================] - 1591s 290ms/step - loss: 1.0194 - accuracy: 0.5506
Epoch 2/4
5494/5494 [==============================] - 1556s 283ms/step - loss: 0.7615 - accuracy: 0.6753
Epoch 3/4
5494/5494 [==============================] - 1556s 283ms/step - loss: 0.6037 - accuracy: 0.7391
Epoch 4/4
5494/5494 [==============================] - 1557s 283ms/step - loss: 0.4840 - accuracy: 0.7846


In [ ]:
model.save_pretrained('/content/drive/MyDrive/roberta_model')

## Fine tuning XLNet with RACE.

In [ ]:
#Limiting the max length of the article given.
max_length = 450

for i in range(len(train_data)):
    article = train_data['article'].iloc[i]
    b = len(article.split())
    if b > max_length:
        c = article.split()
        train_data['article'].iloc[i] = ' '.join(c[:450])

In [ ]:
for i in range(len(dev_data)):
    article = dev_data['article'].iloc[i]
    b = len(article.split())
    if b > max_length:
        c = article.split()
        dev_data['article'].iloc[i] = ' '.join(c[:450])

for i in range(len(test_data)):
    article = test_data['article'].iloc[i]
    b = len(article.split())
    if b > max_length:
        c = article.split()
        test_data['article'].iloc[i] = ' '.join(c[:450])

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('xlnet-base-cased')

In [ ]:
MAX_LEN = 512

In [ ]:
def race_preprocessor(dataset, tokenizer):
    '''
    This function will convert a given article, question, choices in a format:

    article <sep> question choices[0] <sep> <cls>
    article <sep> question choices[1] <sep> <cls>
    article <sep> question choices[2] <sep> <cls>
    article <sep> question choices[3] <sep> <cls>

    After converting in this format the data will be tokenized using a given tokenizer.
    This function will return 4 arrays namely, input_ids, attention_mask, token_type_ids and labels.

    individual input_ids, token_type_ids, attention_mask shape will be as: [num_choices, max_seq_length]
    '''
    all_input_ids = []
    all_attention_mask = []
    all_token_type_ids = []

    for i in range(len(dataset)):

        article = dataset['article'].iloc[i]
        question = dataset['question'].iloc[i]
        options = ast.literal_eval(dataset['options'].iloc[i])
        choice_features = []

        for j in range(len(options)):

            option = options[j]
            input_string = article + '<sep>' + ' ' + question + ' ' + option + '<sep>' + ' ' + '<cls>'
            input_string = re.sub(r'\s+', ' ', input_string)

            input_ids = tokenizer.encode(input_string, add_special_tokens=False, max_length=MAX_LEN)

            padding_length = MAX_LEN - len(input_ids)
            padding_token = tokenizer.pad_token_id

            attention_mask = [1] * len(input_ids)
            
            input_ids = input_ids + [padding_token] * padding_length
            attention_mask = attention_mask + [0] * padding_length
            
            # token_type_ids = inputs['token_type_ids']

            assert len(input_ids) == MAX_LEN
            assert len(attention_mask) == MAX_LEN
            # assert len(token_type_ids) == MAX_LEN

            choice_features.append({'input_ids':input_ids,
                                    'attention_mask':attention_mask})

        all_input_ids.append(np.asarray([cf['input_ids'] for cf in choice_features], dtype='int32'))
        all_attention_mask.append(np.asarray([cf['attention_mask'] for cf in choice_features], dtype='int32'))
        # all_token_type_ids.append(np.asarray([cf['token_type_ids'] for cf in choice_features], dtype='int32'))

    return all_input_ids, all_attention_mask

In [ ]:
dev_input_ids, dev_attention_mask = race_preprocessor(dev_data, tokenizer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
dev_input_ids = np.asarray(dev_input_ids, dtype='int32')
dev_attention_mask = np.asarray(dev_attention_mask, dtype='int32')

In [ ]:
os.mkdir('XLNet files')

In [ ]:
np.save('/content/XLNet files/race_dev_input_ids', dev_input_ids)
np.save('/content/XLNet files/race_dev_attention_mask', dev_attention_mask)

In [ ]:
test_input_ids, test_attention_mask = race_preprocessor(test_data, tokenizer)

In [ ]:
test_input_ids = np.asarray(test_input_ids, dtype='int32')
test_attention_mask = np.asarray(test_attention_mask, dtype='int32')

In [ ]:
np.save('/content/XLNet files/race_test_input_ids', test_input_ids)
np.save('/content/XLNet files/race_test_attention_mask', test_attention_mask)

In [ ]:
train_input_ids, train_attention_mask = race_preprocessor(train_data, tokenizer)

In [ ]:
train_input_ids = np.asarray(train_input_ids, dtype='int32')
train_attention_mask = np.asarray(train_attention_mask, dtype='int32')

In [ ]:
np.save('/content/XLNet files/race_train_input_ids', train_input_ids)
np.save('/content/XLNet files/race_train_attention_mask', train_attention_mask)

In [ ]:
import shutil
shutil.make_archive('XLNet files', 'zip', 'XLNet files')

'/content/XLNet files.zip'

### Training.

In [ ]:
train_input_ids = np.load('/content/drive/MyDrive/RACE files/XLNet files/race_train_input_ids.npy')
train_attention_mask = np.load('/content/drive/MyDrive/RACE files/XLNet files/race_train_attention_mask.npy')

In [ ]:
train_labels = train_data['answerKey'].map(label_map)

In [ ]:
train_labels.values

array([1, 0, 2, ..., 2, 2, 3])

In [ ]:
train_dict = {'input_ids':train_input_ids,
              'attention_mask':train_attention_mask}

In [ ]:
viola = tf.data.Dataset.from_tensor_slices((train_dict,train_labels.values))

In [ ]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
viola = viola.shuffle(10).batch(16).cache().prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
from transformers import TFXLNetForMultipleChoice

with strategy.scope():
    model = TFXLNetForMultipleChoice.from_pretrained('xlnet-base-cased')
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(optimizer=optimizer, 
                  loss=loss, metrics=['accuracy'])

Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetForMultipleChoice: ['lm_loss']
- This IS expected if you are initializing TFXLNetForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFXLNetForMultipleChoice were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj', 'sequence_summary']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
viola

<PrefetchDataset shapes: ({input_ids: (None, 4, 512), attention_mask: (None, 4, 512)}, (None,)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.int64)>

In [ ]:
model.fit(viola, epochs=5)

Epoch 1/5
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


   2/5494 [..............................] - ETA: 31:08:07 - loss: 1.4126 - accuracy: 0.3125WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0132s vs `on_train_batch_end` time: 0.5312s). Check your callbacks.


5494/5494 [==============================] - 3014s 549ms/step - loss: 1.0618 - accuracy: 0.5346
Epoch 2/5
5494/5494 [==============================] - 2974s 541ms/step - loss: 0.7351 - accuracy: 0.6963
Epoch 3/5
5494/5494 [==============================] - 2979s 542ms/step - loss: 0.5258 - accuracy: 0.7827
Epoch 4/5
5494/5494 [==============================] - 2976s 542ms/step - loss: 0.3962 - accuracy: 0.8325
Epoch 5/5
5494/5494 [==============================] - 2978s 542ms/step - loss: 0.3248 - accuracy: 0.8614


In [ ]:
model.save_pretrained('/content/drive/MyDrive/XLNet model')

# ARC dataset.

In [ ]:
df_easy_train = pd.read_csv('/content/drive/My Drive/ARC/df_easy_train_2.csv')
df_easy_dev = pd.read_csv('/content/drive/My Drive/ARC/df_easy_dev_2.csv')
df_easy_test = pd.read_csv('/content/drive/My Drive/ARC/df_easy_test_2.csv')

In [ ]:
df_easy_train.head()

,AnswerKey,question,only_question,quesiton_length,only_answers,context,context_2
0,B,Which factor will most likely cause a person t...,Which factor will most likely cause a person t...,66,(A) a leg muscle relaxing after exercise (B) a...,Piercings show that a person is rebelious and ...,"As a result, the food will not be digested pro..."
1,B,Lichens are symbiotic organisms made of green ...,Lichens are symbiotic organisms made of green ...,139,(A) carbon dioxide (B) food (C) protection (D)...,Lichens are comprised of blue green or green a...,"In this symbiotic relationship, the algae gain..."
2,D,When a switch is used in an electrical circuit...,When a switch is used in an electrical circuit...,63,(A) cause the charge to build. (B) increase an...,Rotary switch A rotary switch is an electric s...,If an ordinary switch is used to control the w...
3,A,Which of the following is an example of an ass...,Which of the following is an example of an ass...,61,(A) contact lens (B) motorcycle (C) raincoat (...,Which of the following is an example of an ang...,The characters are typically members of an out...
4,3,"Rocks are classified as igneous, metamorphic, ...","Rocks are classified as igneous, metamorphic, ...",74,(1) their color (2) their shape (3) how they f...,In one simplistic and linear example elements ...,"In one simplistic and linear example, elements..."


In [ ]:
df_challenge_train = pd.read_csv('/content/drive/My Drive/ARC/df_challenge_train_2.csv')
df_challenge_dev = pd.read_csv('/content/drive/My Drive/ARC/df_challenge_dev_2.csv')
df_challenge_test = pd.read_csv('/content/drive/My Drive/ARC/df_challenge_test_2.csv')

In [ ]:
df_challenge_train.head()

,AnswerKey,question,only_question,quesiton_length,context,only_answers,context_2
0,A,George wants to warm his hands quickly by rubb...,George wants to warm his hands quickly by rubb...,103,The uneven surface reflects heat into other ar...,(A) dry palms (B) wet palms (C) palms covered ...,severe rash with skin swelling including on th...
1,B,Which of the following statements best explain...,Which of the following statements best explain...,98,Which of the following statements best explain...,(A) The refrigerator door is smooth. (B) The r...,He stepped around the refrigerator again and s...
2,B,A fold observed in layers of sedimentary rock ...,A fold observed in layers of sedimentary rock ...,76,A fold in rock layers that forms an arch This ...,(A) cooling of flowing magma. (B) converging o...,Sedimentary rock that forms when minerals crys...
3,D,Which of these do scientists offer as the most...,Which of these do scientists offer as the most...,141,flowering plants Mesozoic Era The Triassic per...,(A) worldwide disease (B) global mountain buil...,During the Cenozoic era larger mammals gave ri...
4,B,A boat is acted on by a river current flowing ...,A boat is acted on by a river current flowing ...,198,The direction in which a skipper wishes to ste...,(A) west (B) east (C) north (D) south,A territory in the north Brazil on the northea...


In [ ]:
def data_generator(a):
    '''
    This function will accept a data point and returns a list of options
    converts the only_options list of options.

    Output will be as follows:

    options = [option_1, option_2, option_3, option_4, option_5]                   

    Note: If there's no option E then it will be written as 'None of the above'.
    '''
    options = []

    if '(A)' in a:
        option_1 = a.split('(B)')[0].replace('(A)','').lstrip()

        b = ' '.join(i for i in a.split() if i not in option_1)
        option_2 = b.split('(C)')[0].replace('(B)','').replace('(A)','').lstrip()

        c = ' '.join(i for i in b.split() if i not in option_2)
        option_3 = c.split('(D)')[0].replace('(C)','').replace('(B)','').replace('(A)','').lstrip()

        if '(D)' not in c:
            option_4 = 'None of the above'
            option_5 = 'None of the above'
        else:
            if '(E)' not in c:
                option_4 = c.split('(D)')[1].lstrip()
                option_5 = 'None of the above'
            else:
                d = ' '.join(i for i in c.split() if i not in option_3)
                option_4 = d.split('(E)')[0].replace('(D)','').replace('(C)','').replace('(B)','').replace('(A)','').lstrip()
                option_5 = d.split('(E)')[1].lstrip()
    else:
        option_1 = a.split('(2)')[0].replace('(1)','').lstrip()

        b = ' '.join(i for i in a.split() if i not in option_1)
        option_2 = b.split('(3)')[0].replace('(2)','').replace('(1)','').lstrip()

        c = ' '.join(i for i in b.split() if i not in option_2)
        option_3 = c.split('(4)')[0].replace('(3)','').replace('(2)','').replace('(1)','').lstrip()

        if '(4)' in c:
            option_4 = c.split('(4)')[1].lstrip()
        else:
            option_4 = 'None of the above'
        option_5 = 'None of the above'

    options = [option_1, option_2, option_3, option_4, option_5]

    return options

In [ ]:
df_easy_dev['options_list'] = df_easy_dev['only_answers'].map(data_generator)
df_easy_train['options_list'] = df_easy_train['only_answers'].map(data_generator)
df_easy_test['options_list'] = df_easy_test['only_answers'].map(data_generator)

In [ ]:
def answer(answer_key, options_list):

    label_map = {'A':0,
             'B':1,
             'C':2,
             'D':3,
             'E':4,
             '1':0,
             '2':1,
             '3':2,
             '4':3,
             '5':4}
    
    number = label_map.get(answer_key)
    answer = options_list[number]
    return answer

In [ ]:
df_easy_train['Answer'] = df_easy_train[['AnswerKey', 'options_list']].apply(lambda x: answer(*x), axis=1)
df_easy_dev['Answer'] = df_easy_dev[['AnswerKey', 'options_list']].apply(lambda x: answer(*x), axis=1)
df_easy_test['Answer'] = df_easy_test[['AnswerKey', 'options_list']].apply(lambda x: answer(*x), axis=1)

In [ ]:
label_map = {'A':0,
             'B':1,
             'C':2,
             'D':3,
             'E':4,
             '1':0,
             '2':1,
             '3':2,
             '4':3,
             '5':4}

easy_train_labels = df_easy_train['AnswerKey'].map(label_map)
easy_dev_labels = df_easy_dev['AnswerKey'].map(label_map)
easy_test_labels = df_easy_test['AnswerKey'].map(label_map)

In [ ]:
df_challenge_dev['options_list'] = df_challenge_dev['only_answers'].map(data_generator)
df_challenge_train['options_list'] = df_challenge_train['only_answers'].map(data_generator)
df_challenge_test['options_list'] = df_challenge_test['only_answers'].map(data_generator)

In [ ]:
label_map = {'A':0,
             'B':1,
             'C':2,
             'D':3,
             'E':4,
             '1':0,
             '2':1,
             '3':2,
             '4':3,
             '5':4}

challenge_train_labels = df_challenge_train['AnswerKey'].map(label_map)
challenge_dev_labels = df_challenge_dev['AnswerKey'].map(label_map)
challenge_test_labels = df_challenge_test['AnswerKey'].map(label_map)

## Baseline Model.

For base model we will select answers randomly.

In [ ]:
char_list_1 = ['A','B','C','D']
char_list_2 = ['A','B','C','D','E']
num_list = ['1','2','3','4']

In [ ]:
easy_train_preds = []
easy_dev_preds = []
easy_test_preds = []

for i in range(len(df_easy_train)):
    if '(1)' in df_easy_train['only_answers'].iloc[i]:
        easy_train_preds.append(random.choice(num_list))
    else:
        if '(E)' in df_easy_train['only_answers'].iloc[i]:
            easy_train_preds.append(random.choice(char_list_2))
        else:
            easy_train_preds.append(random.choice(char_list_1))

In [ ]:
for i in range(len(df_easy_dev)):
    if '(1)' in df_easy_dev['only_answers'].iloc[i]:
        easy_dev_preds.append(random.choice(num_list))
    else:
        if '(E)' in df_easy_dev['only_answers'].iloc[i]:
            easy_dev_preds.append(random.choice(char_list_2))
        else:
            easy_dev_preds.append(random.choice(char_list_1))

In [ ]:
for i in range(len(df_easy_test)):
    if '(1)' in df_easy_test['only_answers'].iloc[i]:
        easy_test_preds.append(random.choice(num_list))
    else:
        if '(E)' in df_easy_test['only_answers'].iloc[i]:
            easy_test_preds.append(random.choice(char_list_2))
        else:
            easy_test_preds.append(random.choice(char_list_1))

In [ ]:
challenge_train_preds = []
challenge_dev_preds = []
challenge_test_preds = []

for i in range(len(df_challenge_train)):
    if '(1)' in df_challenge_train['only_answers'].iloc[i]:
        challenge_train_preds.append(random.choice(num_list))
    else:
        if '(E)' in df_challenge_train['only_answers'].iloc[i]:
            challenge_train_preds.append(random.choice(char_list_2))
        else:
            challenge_train_preds.append(random.choice(char_list_1))

for i in range(len(df_challenge_dev)):
    if '(1)' in df_challenge_dev['only_answers'].iloc[i]:
        challenge_dev_preds.append(random.choice(num_list))
    else:
        if '(E)' in df_challenge_dev['only_answers'].iloc[i]:
            challenge_dev_preds.append(random.choice(char_list_2))
        else:
            challenge_dev_preds.append(random.choice(char_list_1))

for i in range(len(df_challenge_test)):
    if '(1)' in df_challenge_test['only_answers'].iloc[i]:
        challenge_test_preds.append(random.choice(num_list))
    else:
        if '(E)' in df_challenge_test['only_answers'].iloc[i]:
            challenge_test_preds.append(random.choice(char_list_2))
        else:
            challenge_test_preds.append(random.choice(char_list_1))

In [ ]:
easy_train_score = accuracy_score(df_easy_train['AnswerKey'], easy_train_preds)
easy_dev_score = accuracy_score(df_easy_dev['AnswerKey'], easy_dev_preds)
easy_test_score = accuracy_score(df_easy_test['AnswerKey'], easy_test_preds)

print(f'Accuracy score on easy train set is {easy_train_score}.')
print(f'Accuracy score on easy dev set is {easy_dev_score}.')
print(f'Accuracy score on easy test set is {easy_test_score}.')

In [ ]:
challenge_train_score = accuracy_score(df_challenge_train['AnswerKey'], challenge_train_preds)
challenge_dev_score = accuracy_score(df_challenge_dev['AnswerKey'], challenge_dev_preds)
challenge_test_score = accuracy_score(df_challenge_test['AnswerKey'], challenge_test_preds)

print(f'Accuracy score on challenge train set is {challenge_train_score}.')
print(f'Accuracy score on challenge dev set is {challenge_dev_score}.')
print(f'Accuracy score on challenge test set is {challenge_test_score}.')

## BERT Model.

### Preparing data according to BERT.

In [ ]:
from transformers import BertTokenizer, TFBertForMultipleChoice

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
MAX_LEN = 512

In [ ]:
def arc_preprocessor(dataset, tokenizer):
    '''
    This function will convert a given context, question, choices in a format:

    [CLS] context [SEP] question choices[0] [SEP]
    [CLS] context [SEP] question choices[1] [SEP]
    [CLS] context [SEP] question choices[2] [SEP]
    [CLS] context [SEP] question choices[3] [SEP]

    After converting in this format the data will be tokenized using a given tokenizer.
    This function will return 4 arrays namely, input_ids, attention_mask, token_type_ids and labels.

    individual input_ids, token_type_ids, attention_mask shape will be as: [num_choices, max_seq_length]
    '''
    all_input_ids = []
    all_attention_mask = []
    all_token_type_ids = []

    for i in range(len(dataset)):

        context = dataset['context_2'].iloc[i]
        question = dataset['only_question'].iloc[i]
        options = dataset['options_list'].iloc[i]
        choice_features = []

        for j in range(len(options)):

            option = options[j]
            ques_opt = question + ' ' + f'{j}' + ' ' + option
            ques_opt = re.sub(r'\s+', ' ', ques_opt)

            inputs = tokenizer(context,
                               ques_opt,
                               max_length=MAX_LEN,
                               truncation='only_first',
                               padding='max_length')
            
            input_ids = inputs['input_ids']
            attention_mask = inputs['attention_mask']
            token_type_ids = inputs['token_type_ids']

            assert len(input_ids) == MAX_LEN
            assert len(token_type_ids) == MAX_LEN
            assert len(attention_mask) == MAX_LEN

            choice_features.append({'input_ids':input_ids,
                                    'attention_mask':attention_mask,
                                    'token_type_ids':token_type_ids})

        all_input_ids.append(np.asarray([cf['input_ids'] for cf in choice_features], dtype='int32'))
        all_attention_mask.append(np.asarray([cf['attention_mask'] for cf in choice_features], dtype='int32'))
        all_token_type_ids.append(np.asarray([cf['token_type_ids'] for cf in choice_features], dtype='int32'))

    return all_input_ids, all_attention_mask, all_token_type_ids

In [ ]:
easy_train_input_ids, easy_train_attention_mask, easy_train_token_type_ids = arc_preprocessor(df_easy_train, tokenizer)
easy_dev_input_ids, easy_dev_attention_mask, easy_dev_token_type_ids = arc_preprocessor(df_easy_dev, tokenizer)
easy_test_input_ids, easy_test_attention_mask, easy_test_token_type_ids = arc_preprocessor(df_easy_test, tokenizer)

In [ ]:
easy_train_input_ids = np.asarray(easy_train_input_ids, dtype='int32')
easy_train_token_type_ids = np.asarray(easy_train_token_type_ids, dtype='int32')
easy_train_attention_mask = np.asarray(easy_train_attention_mask, dtype='int32')

In [ ]:
easy_dev_input_ids = np.asarray(easy_dev_input_ids, dtype='int32')
easy_dev_token_type_ids = np.asarray(easy_dev_token_type_ids, dtype='int32')
easy_dev_attention_mask = np.asarray(easy_dev_attention_mask, dtype='int32')

In [ ]:
easy_test_input_ids = np.asarray(easy_test_input_ids, dtype='int32')
easy_test_token_type_ids = np.asarray(easy_test_token_type_ids, dtype='int32')
easy_test_attention_mask = np.asarray(easy_test_attention_mask, dtype='int32')

In [ ]:
np.save('/content/drive/MyDrive/ARC/BERT files/easy_train_input_ids', easy_train_input_ids)
np.save('/content/drive/MyDrive/ARC/BERT files/easy_train_attention_mask', easy_train_attention_mask)
np.save('/content/drive/MyDrive/ARC/BERT files/easy_train_token_type_ids', easy_train_token_type_ids)

In [ ]:
np.save('/content/drive/MyDrive/ARC/BERT files/easy_dev_input_ids', easy_dev_input_ids)
np.save('/content/drive/MyDrive/ARC/BERT files/easy_dev_attention_mask', easy_dev_attention_mask)
np.save('/content/drive/MyDrive/ARC/BERT files/easy_dev_token_type_ids', easy_dev_token_type_ids)

In [ ]:
np.save('/content/drive/MyDrive/ARC/BERT files/easy_test_input_ids', easy_test_input_ids)
np.save('/content/drive/MyDrive/ARC/BERT files/easy_test_attention_mask', easy_test_attention_mask)
np.save('/content/drive/MyDrive/ARC/BERT files/easy_test_token_type_ids', easy_test_token_type_ids)

### Training BERT.

In [ ]:
easy_dev_input_ids = np.load('/content/drive/MyDrive/ARC/BERT files/easy_dev_input_ids.npy')
easy_dev_attention_mask = np.load('/content/drive/MyDrive/ARC/BERT files/easy_dev_attention_mask.npy')
easy_dev_token_type_ids = np.load('/content/drive/MyDrive/ARC/BERT files/easy_dev_token_type_ids.npy')

In [ ]:
easy_test_input_ids = np.load('/content/drive/MyDrive/ARC/BERT files/easy_test_input_ids.npy')
easy_test_attention_mask = np.load('/content/drive/MyDrive/ARC/BERT files/easy_test_attention_mask.npy')
easy_test_token_type_ids = np.load('/content/drive/MyDrive/ARC/BERT files/easy_test_token_type_ids.npy')

In [ ]:
easy_train_input_ids = np.load('/content/drive/MyDrive/ARC/BERT files/easy_train_input_ids.npy')
easy_train_attention_mask = np.load('/content/drive/MyDrive/ARC/BERT files/easy_train_attention_mask.npy')
easy_train_token_type_ids = np.load('/content/drive/MyDrive/ARC/BERT files/easy_train_token_type_ids.npy')

In [ ]:
pre_trained_model = TFBertForMultipleChoice.from_pretrained('/content/drive/MyDrive/race_bert model', return_dict=True)

model_input_ids = Input(shape=(5,512,), name='input_tokens', dtype='int32')
masks_input = Input(shape=(5,512,), name='attention_mask', dtype='int32')
model_token_type_ids = Input(shape=(5,512,), name='token_type_ids', dtype='int32')

x = {'input_ids':model_input_ids,
        'attention_mask':masks_input,
        'token_type_ids':model_token_type_ids}

x = pre_trained_model(x)['logits']
outputs = Dense(5, activation='softmax')(x)

model = Model(inputs=[model_input_ids, masks_input,model_token_type_ids], outputs=outputs)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.CategoricalCrossentropy()
model.compile(optimizer=optimizer, 
                loss=loss, metrics=['accuracy'])

All model checkpoint layers were used when initializing TFBertForMultipleChoice.

All the layers of TFBertForMultipleChoice were initialized from the model checkpoint at /content/drive/MyDrive/race_bert model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMultipleChoice for predictions without further training.


In [ ]:
early_stop = EarlyStopping(patience=2, monitor='val_accuracy')

model.fit(x=[easy_train_input_ids, easy_train_attention_mask, easy_train_token_type_ids], y=tf.keras.utils.to_categorical(easy_train_labels.values),
          epochs=4, batch_size=2)

Epoch 1/4
1126/1126 [==============================] - 1427s 1s/step - loss: 1.2875 - accuracy: 0.3980
Epoch 2/4
1126/1126 [==============================] - 1425s 1s/step - loss: 0.8313 - accuracy: 0.6242
Epoch 3/4
1126/1126 [==============================] - 1427s 1s/step - loss: 0.5845 - accuracy: 0.7579
Epoch 4/4
1126/1126 [==============================] - 1426s 1s/step - loss: 0.4036 - accuracy: 0.8423


In [ ]:
model.save('/content/drive/MyDrive/ARC/bert_model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/MyDrive/ARC/bert_model/assets


In [ ]:
model.save_weights('/content/drive/MyDrive/ARC/bert_weights')

In [ ]:
model.load_weights('/content/drive/MyDrive/ARC/bert weights/bert_weights')

In [ ]:
dev_preds = model.predict([easy_dev_input_ids,easy_dev_attention_mask,easy_dev_token_type_ids])
dev_preds_max = tf.argmax(dev_preds, axis=1)

In [ ]:
print(f'Accuracy score on easy dev set is {accuracy_score(easy_dev_labels,dev_preds_max)}.')

Accuracy score on easy dev set is 0.3263157894736842.


In [ ]:
test_preds = model.predict([easy_test_input_ids,easy_test_attention_mask,easy_test_token_type_ids])
test_preds_max = tf.argmax(test_preds, axis=1)

In [ ]:
print(f'Accuracy score on easy test set is {accuracy_score(easy_test_labels,test_preds_max)}.')

Accuracy score on easy test set is 0.33080808080808083.


## RoBERTa Model.

In [ ]:
from transformers import RobertaTokenizer, TFRobertaForMultipleChoice

### Preparing data according to RoBERTa.

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)

In [ ]:
MAX_LEN=512

In [ ]:
def arc_preprocessor(dataset, tokenizer):
    '''
    This function will convert a given article, question, choices in a format:

    <s> article </s> </s> question </s> </s> choices[0] </s>
    <s> article </s> </s> question </s> </s> choices[1] </s>
    <s> article </s> </s> question </s> </s> choices[2] </s>
    <s> article </s> </s> question </s> </s> choices[3] </s>

    After converting in this format the data will be tokenized using a given tokenizer.
    This function will return 4 arrays namely, input_ids, attention_mask, token_type_ids and labels.

    individual input_ids, token_type_ids, attention_mask shape will be as: [num_choices, max_seq_length]
    '''
    global MAX_LEN
    all_input_ids = []
    all_attention_mask = []
    all_token_type_ids = []

    for i in range(len(dataset)):
        
        context = ' '.join(dataset['context_2'].iloc[i].split()[:400]) #Limiting the maximum length of context to 400 words.
        question = dataset['only_question'].iloc[i]
        options = dataset['options_list'].iloc[i]
        choice_features = []

        for j in range(len(options)):

            option = options[j]
            input_string = '<s>' + ' ' + context + ' ' + '</s>' + ' ' + '</s>' + ' ' + question + ' ' + '</s>' + ' ' + '</s>' + ' ' + option + ' ' + '</s>'
            input_string = re.sub(r'\s+', ' ', input_string)

            input_ids = tokenizer(input_string,
                               max_length=MAX_LEN,
                               add_special_tokens=False, truncation='only_first')['input_ids']
            
            attention_mask = [1] * len(input_ids)

            padding_id = tokenizer.pad_token_id
            padding_length = MAX_LEN - len(input_ids)

            input_ids = input_ids + [padding_id]*padding_length
            attention_mask = attention_mask + [0]*padding_length

            assert len(input_ids) == MAX_LEN
            assert len(attention_mask) == MAX_LEN

            choice_features.append({'input_ids':input_ids,
                                    'attention_mask':attention_mask})

        all_input_ids.append(np.asarray([cf['input_ids'] for cf in choice_features], dtype='int32'))
        all_attention_mask.append(np.asarray([cf['attention_mask'] for cf in choice_features], dtype='int32'))

    return all_input_ids, all_attention_mask

In [ ]:
easy_train_input_ids, easy_train_attention_mask = arc_preprocessor(df_easy_train, tokenizer)
easy_dev_input_ids, easy_dev_attention_mask = arc_preprocessor(df_easy_dev, tokenizer)
easy_test_input_ids, easy_test_attention_mask = arc_preprocessor(df_easy_test, tokenizer)

In [ ]:
easy_train_input_ids = np.asarray(easy_train_input_ids, dtype='int32')
easy_train_attention_mask = np.asarray(easy_train_attention_mask, dtype='int32')

In [ ]:
easy_dev_input_ids = np.asarray(easy_dev_input_ids, dtype='int32')
easy_dev_attention_mask = np.asarray(easy_dev_attention_mask, dtype='int32')

In [ ]:
easy_test_input_ids = np.asarray(easy_test_input_ids, dtype='int32')
easy_test_attention_mask = np.asarray(easy_test_attention_mask, dtype='int32')

In [ ]:
np.save('/content/drive/MyDrive/ARC/RoBERTa_2 files/easy_train_input_ids', easy_train_input_ids)
np.save('/content/drive/MyDrive/ARC/RoBERTa_2 files/easy_train_attention_mask', easy_train_attention_mask)

In [ ]:
np.save('/content/drive/MyDrive/ARC/RoBERTa_2 files/easy_dev_input_ids', easy_dev_input_ids)
np.save('/content/drive/MyDrive/ARC/RoBERTa_2 files/easy_dev_attention_mask', easy_dev_attention_mask)

In [ ]:
np.save('/content/drive/MyDrive/ARC/RoBERTa_2 files/easy_test_input_ids', easy_test_input_ids)
np.save('/content/drive/MyDrive/ARC/RoBERTa_2 files/easy_test_attention_mask', easy_test_attention_mask)

### Training RoBERTa.

In [ ]:
easy_train_input_ids = np.load('/content/drive/MyDrive/ARC/RoBERTa_2 files/easy_train_input_ids.npy')
easy_train_attention_mask = np.load('/content/drive/MyDrive/ARC/RoBERTa_2 files/easy_train_attention_mask.npy')

In [ ]:
easy_dev_input_ids = np.load('/content/drive/MyDrive/ARC/RoBERTa_2 files/easy_dev_input_ids.npy')
easy_dev_attention_mask = np.load('/content/drive/MyDrive/ARC/RoBERTa_2 files/easy_dev_attention_mask.npy')

In [ ]:
easy_test_input_ids = np.load('/content/drive/MyDrive/ARC/RoBERTa_2 files/easy_test_input_ids.npy')
easy_test_attention_mask = np.load('/content/drive/MyDrive/ARC/RoBERTa_2 files/easy_test_attention_mask.npy')

In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
with strategy.scope():
    pre_trained_model = TFRobertaForMultipleChoice.from_pretrained('/content/drive/MyDrive/roberta_model', return_dict=True)

    model_input_ids = Input(shape=(5,512,), name='input_tokens', dtype='int32')
    masks_input = Input(shape=(5,512,), name='attention_mask', dtype='int32')

    x =  {'input_ids':model_input_ids,
            'attention_mask':masks_input}

    x = pre_trained_model(x)['logits']
    outputs = Dense(5, activation='softmax')(x)

    model = Model(inputs=[model_input_ids, masks_input], outputs=outputs)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
valid_y = tf.keras.utils.to_categorical(easy_dev_labels, num_classes=5)

In [ ]:
model.fit(x=[easy_train_input_ids, easy_train_attention_mask], y=tf.keras.utils.to_categorical(easy_train_labels),
          batch_size=8, epochs=5,
          validation_data=([easy_dev_input_ids, easy_dev_attention_mask], valid_y),
          validation_batch_size=8)

Epoch 1/5
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


  2/282 [..............................] - ETA: 1:32:15 - loss: 3.5207 - accuracy: 0.0000e+00WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0112s vs `on_train_batch_end` time: 0.3293s). Check your callbacks.


282/282 [==============================] - ETA: 0s - loss: 1.6328 - accuracy: 0.2155WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0039s vs `on_test_batch_end` time: 0.0912s). Check your callbacks.


282/282 [==============================] - 152s 540ms/step - loss: 1.6328 - accuracy: 0.2155 - val_loss: 1.6083 - val_accuracy: 0.2719
Epoch 2/5
282/282 [==============================] - 102s 361ms/step - loss: 1.6115 - accuracy: 0.2350 - val_loss: 1.6070 - val_accuracy: 0.2561
Epoch 3/5
282/282 [==============================] - 102s 361ms/step - loss: 1.6096 - accuracy: 0.2439 - val_loss: 1.6107 - val_accuracy: 0.2649
Epoch 4/5
282/282 [==============================] - 102s 362ms/step - loss: 1.6017 - accuracy: 0.2612 - val_loss: 1.6383 - val_accuracy: 0.2474
Epoch 5/5
282/282 [==============================] - 102s 362ms/step - loss: 1.5941 - accuracy: 0.2803 - val_loss: 1.6233 - val_accuracy: 0.2421


In [ ]:
model.save_weights('/content/drive/MyDrive/ARC/roberta weights/roberta_weights')

In [ ]:
# model.load_weights('/content/drive/MyDrive/ARC/roberta weights/roberta_weights')

In [ ]:
test_preds = model.predict([easy_test_input_ids,easy_test_attention_mask])
test_preds_max = tf.argmax(test_preds, axis=1)

In [ ]:
print(f'Accuracy score on easy test set is {accuracy_score(easy_test_labels,test_preds_max)}.')

Accuracy score on easy test set is 0.255050505050505.


## XLNET Model.

### Preparing data according to XLNET.

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)

In [ ]:
MAX_LEN=512

In [ ]:
def arc_preprocessor(dataset, tokenizer):
    '''
    This function will convert a given article, question, choices in a format:

    article <sep> question choices[0] <sep> <cls>
    article <sep> question choices[1] <sep> <cls>
    article <sep> question choices[2] <sep> <cls>
    article <sep> question choices[3] <sep> <cls>

    After converting in this format the data will be tokenized using a given tokenizer.
    This function will return 4 arrays namely, input_ids, attention_mask, token_type_ids and labels.

    individual input_ids, token_type_ids, attention_mask shape will be as: [num_choices, max_seq_length]
    '''
    global MAX_LEN
    all_input_ids = []
    all_attention_mask = []
    all_token_type_ids = []

    for i in range(len(dataset)):
        
        context = ' '.join(dataset['context_2'].iloc[i].split()[:400]) #Limiting the maximum length of context to 400 words.
        question = dataset['only_question'].iloc[i]
        options = dataset['options_list'].iloc[i]
        choice_features = []

        for j in range(len(options)):

            option = options[j]
            input_string = '<s>' + ' ' + context + ' ' + '</s>' + ' ' + question + ' ' + '</s>' + ' ' + option + ' ' + '</s>'
            input_string = re.sub(r'\s+', ' ', input_string)

            input_ids = tokenizer(input_string,
                               max_length=MAX_LEN,
                               add_special_tokens=False)['input_ids']
            
            attention_mask = [1] * len(input_ids)

            padding_id = tokenizer.pad_token_id
            padding_length = MAX_LEN - len(input_ids)

            input_ids = input_ids + [padding_id]*padding_length
            attention_mask = attention_mask + [0]*padding_length

            assert len(input_ids) == MAX_LEN
            assert len(attention_mask) == MAX_LEN

            choice_features.append({'input_ids':input_ids,
                                    'attention_mask':attention_mask})

        all_input_ids.append(np.asarray([cf['input_ids'] for cf in choice_features], dtype='int32'))
        all_attention_mask.append(np.asarray([cf['attention_mask'] for cf in choice_features], dtype='int32'))

    return all_input_ids, all_attention_mask

In [ ]:
easy_train_input_ids, easy_train_attention_mask = arc_preprocessor(df_easy_train, tokenizer)
easy_dev_input_ids, easy_dev_attention_mask = arc_preprocessor(df_easy_dev, tokenizer)
easy_test_input_ids, easy_test_attention_mask = arc_preprocessor(df_easy_test, tokenizer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
easy_train_input_ids = np.asarray(easy_train_input_ids, dtype='int32')
easy_train_attention_mask = np.asarray(easy_train_attention_mask, dtype='int32')

In [ ]:
easy_dev_input_ids = np.asarray(easy_dev_input_ids, dtype='int32')
easy_dev_attention_mask = np.asarray(easy_dev_attention_mask, dtype='int32')

In [ ]:
easy_test_input_ids = np.asarray(easy_test_input_ids, dtype='int32')
easy_test_attention_mask = np.asarray(easy_test_attention_mask, dtype='int32')

In [ ]:
np.save('/content/drive/MyDrive/ARC/XLNet files/easy_train_input_ids', easy_train_input_ids)
np.save('/content/drive/MyDrive/ARC/XLNet files/easy_train_attention_mask', easy_train_attention_mask)

In [ ]:
np.save('/content/drive/MyDrive/ARC/XLNet files/easy_dev_input_ids', easy_dev_input_ids)
np.save('/content/drive/MyDrive/ARC/XLNet files/easy_dev_attention_mask', easy_dev_attention_mask)

In [ ]:
np.save('/content/drive/MyDrive/ARC/XLNet files/easy_test_input_ids', easy_test_input_ids)
np.save('/content/drive/MyDrive/ARC/XLNet files/easy_test_attention_mask', easy_test_attention_mask)

### Training XLNet.

In [ ]:
easy_train_input_ids = np.load('/content/drive/MyDrive/ARC/XLNet files/easy_train_input_ids.npy')
easy_train_attention_mask = np.load('/content/drive/MyDrive/ARC/XLNet files/easy_train_attention_mask.npy')

In [ ]:
easy_dev_input_ids = np.load('/content/drive/MyDrive/ARC/XLNet files/easy_dev_input_ids.npy')
easy_dev_attention_mask = np.load('/content/drive/MyDrive/ARC/XLNet files/easy_dev_attention_mask.npy')

In [ ]:
easy_test_input_ids = np.load('/content/drive/MyDrive/ARC/XLNet files/easy_test_input_ids.npy')
easy_test_attention_mask = np.load('/content/drive/MyDrive/ARC/XLNet files/easy_test_attention_mask.npy')

In [ ]:
from transformers import TFXLNetForMultipleChoice

In [ ]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
easy_train_dict = {'input_ids':easy_train_input_ids,
                   'attention_mask':easy_train_attention_mask}

viola = tf.data.Dataset.from_tensor_slices((easy_train_dict,tf.keras.utils.to_categorical(easy_train_labels.values)))

In [ ]:
viola = viola.shuffle(32).batch(8).cache().prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
easy_dev_dict = {'input_ids':easy_dev_input_ids,
                   'attention_mask':easy_dev_attention_mask}

viola_dev = tf.data.Dataset.from_tensor_slices((easy_dev_dict,tf.keras.utils.to_categorical(easy_dev_labels.values, num_classes=5)))

In [ ]:
viola_dev = viola_dev.shuffle(32).batch(8).cache().prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
with strategy.scope():
    pre_trained_model = TFXLNetForMultipleChoice.from_pretrained('/content/drive/MyDrive/XLNet model', return_dict=True)

    model_input_ids = Input(shape=(5,512,), name='input_tokens', dtype='int32')
    masks_input = Input(shape=(5,512,), name='attention_mask', dtype='int32')

    x =  {'input_ids':model_input_ids,
            'attention_mask':masks_input}

    x = pre_trained_model(x)['logits']
    # x = Dense(64, activation='relu', kernel_initializer='he_normal')(x)
    # x = Dropout(0.2)(x)
    outputs = Dense(5, activation='softmax')(x)

    model = Model(inputs=[model_input_ids, masks_input], outputs=outputs)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-6),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])

InvalidArgumentError: ignored

In [ ]:
model.fit(viola, epochs=5)

Epoch 1/5
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


  2/282 [..............................] - ETA: 1:45:49 - loss: 7.7513 - accuracy: 0.1250WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0093s vs `on_train_batch_end` time: 0.7045s). Check your callbacks.


282/282 [==============================] - 246s 874ms/step - loss: 1.8952 - accuracy: 0.2177
Epoch 2/5
282/282 [==============================] - 203s 719ms/step - loss: 1.6157 - accuracy: 0.2399
Epoch 3/5
282/282 [==============================] - 203s 719ms/step - loss: 1.6113 - accuracy: 0.2319
Epoch 4/5
282/282 [==============================] - 203s 719ms/step - loss: 1.5863 - accuracy: 0.2394
Epoch 5/5
282/282 [==============================] - 203s 719ms/step - loss: 1.5850 - accuracy: 0.2443


In [ ]:
model.save_weights('/content/drive/MyDrive/ARC/xlnet weights/')

In [ ]:
# model.load_weights('/content/drive/MyDrive/ARC/xlnet weights')

In [ ]:
dev_preds = model.predict([easy_dev_input_ids,easy_dev_attention_mask])
dev_preds_max = tf.argmax(dev_preds, axis=1)

In [ ]:
print(f'Accuracy score on easy dev set is {accuracy_score(easy_dev_labels,dev_preds_max)}.')

Accuracy score on easy dev set is 0.21754385964912282.


In [ ]:
test_preds = model.predict([easy_test_input_ids,easy_test_attention_mask])
test_preds_max = tf.argmax(test_preds, axis=1)

In [ ]:
print(f'Accuracy score on easy test set is {accuracy_score(easy_test_labels,test_preds_max)}.')

Accuracy score on easy test set is 0.24116161616161616.


# Fine tuning Unified QA on ARC without using context.

### Preparing Data.

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained('t5-small')

In [ ]:
MAX_LEN = 512
decoder_max_len = 30

In [ ]:
def arc_preprocessor(dataset, tokenizer):
    '''
    This function will convert a given context, question, choices in a format:

    input: question \n options \n context </s>
    target: label </s>

    After converting in this format the data will be tokenized using a given tokenizer.
    This function will return 4 arrays namely, input_ids, attention_mask, token_type_ids and labels.
    '''
    global MAX_LEN
    all_input_ids = []
    all_attention_mask = []
    all_decoder_input_ids = []
    all_labels = []

    for i in range(len(dataset)):
        
        context = ' '.join(dataset['context_2'].iloc[i].split()[:350]) #Limiting the maximum length of context to 350 words.
        question = dataset['only_question'].iloc[i]
        options = dataset['only_answers'].iloc[i]
        target = dataset['Answer'].iloc[i]
        choice_features = []

        input_string = question + ' ' + '\\n' + ' ' + options + ' ' + '</s>'
        decoder_input = tokenizer.pad_token + ' ' + target
        target = target + ' ' + '</s>'

        input_ids = tokenizer.encode(input_string, truncation=True, max_length=MAX_LEN)
        decoder_input_ids = tokenizer.encode(decoder_input, max_length=decoder_max_len, truncation=True) #Max length of a answer is 23
        labels = tokenizer.encode(target, max_length=decoder_max_len, truncation=True)

        attention_mask = [1] * len(input_ids)

        padding_id = tokenizer.pad_token_id
        padding_length = MAX_LEN - len(input_ids)

        input_ids = input_ids + [padding_id]*padding_length
        attention_mask = attention_mask + [0]*padding_length

        deocder_padding_length = decoder_max_len - len(decoder_input_ids)

        decoder_input_ids = decoder_input_ids + [padding_id]*deocder_padding_length
        labels = labels + [padding_id]*deocder_padding_length

        assert len(input_ids) == MAX_LEN
        assert len(attention_mask) == MAX_LEN

        all_input_ids.append(np.asarray(input_ids, dtype='int32'))
        all_attention_mask.append(np.asarray(attention_mask, dtype='int32'))
        all_decoder_input_ids.append(np.asarray(decoder_input_ids, dtype='int32'))
        all_labels.append(np.asarray(labels, dtype='int32'))

    return all_input_ids, all_attention_mask, all_decoder_input_ids, all_labels

In [ ]:
easy_train_input_ids, easy_train_attention_mask, easy_train_decoder_input_ids, easy_train_labels = arc_preprocessor(df_easy_train, tokenizer)

In [ ]:
easy_dev_input_ids, easy_dev_attention_mask, easy_dev_decoder_input_ids, easy_dev_labels = arc_preprocessor(df_easy_dev, tokenizer)

In [ ]:
easy_test_input_ids, easy_test_attention_mask, _, easy_test_labels = arc_preprocessor(df_easy_test, tokenizer)

In [ ]:
easy_train_input_ids = np.asarray(easy_train_input_ids, dtype='int32')
easy_train_attention_mask = np.asarray(easy_train_attention_mask, dtype='int32')
easy_train_decoder_input_ids = np.asarray(easy_train_decoder_input_ids, dtype='int32')
easy_train_labels = np.asarray(easy_train_labels, dtype='int32')

In [ ]:
easy_dev_input_ids = np.asarray(easy_dev_input_ids, dtype='int32')
easy_dev_attention_mask = np.asarray(easy_dev_attention_mask, dtype='int32')
easy_dev_decoder_input_ids = np.asarray(easy_dev_decoder_input_ids, dtype='int32')
easy_dev_labels = np.asarray(easy_dev_labels, dtype='int32')

In [ ]:
easy_test_input_ids = np.asarray(easy_test_input_ids, dtype='int32')
easy_test_attention_mask = np.asarray(easy_test_attention_mask, dtype='int32')
easy_test_labels = np.asarray(easy_test_labels, dtype='int32')

In [ ]:
np.save('/content/drive/MyDrive/ARC/Unified QA/easy_train_input_ids', easy_train_input_ids)
np.save('/content/drive/MyDrive/ARC/Unified QA/easy_train_attention_mask', easy_train_attention_mask)
np.save('/content/drive/MyDrive/ARC/Unified QA/easy_train_decoder_input_ids', easy_train_decoder_input_ids)
np.save('/content/drive/MyDrive/ARC/Unified QA/easy_train_labels', easy_train_labels)

In [ ]:
np.save('/content/drive/MyDrive/ARC/Unified QA/easy_dev_input_ids', easy_dev_input_ids)
np.save('/content/drive/MyDrive/ARC/Unified QA/easy_dev_attention_mask', easy_dev_attention_mask)
np.save('/content/drive/MyDrive/ARC/Unified QA/easy_dev_decoder_input_ids', easy_dev_decoder_input_ids)
np.save('/content/drive/MyDrive/ARC/Unified QA/easy_dev_labels', easy_dev_labels)

In [ ]:
np.save('/content/drive/MyDrive/ARC/Unified QA/easy_test_input_ids', easy_test_input_ids)
np.save('/content/drive/MyDrive/ARC/Unified QA/easy_test_attention_mask', easy_test_attention_mask)
np.save('/content/drive/MyDrive/ARC/Unified QA/easy_test_labels', easy_test_labels)

### Training.

In [ ]:
easy_train_input_ids = np.load('/content/drive/MyDrive/ARC/Unified QA/easy_train_input_ids.npy')
easy_train_attention_mask = np.load('/content/drive/MyDrive/ARC/Unified QA/easy_train_attention_mask.npy')
easy_train_decoder_input_ids = np.load('/content/drive/MyDrive/ARC/Unified QA/easy_train_decoder_input_ids.npy')
easy_train_labels = np.load('/content/drive/MyDrive/ARC/Unified QA/easy_train_labels.npy')

In [ ]:
easy_dev_input_ids = np.load('/content/drive/MyDrive/ARC/Unified QA/easy_dev_input_ids.npy')
easy_dev_attention_mask = np.load('/content/drive/MyDrive/ARC/Unified QA/easy_dev_attention_mask.npy')
easy_dev_decoder_input_ids = np.load('/content/drive/MyDrive/ARC/Unified QA/easy_dev_decoder_input_ids.npy')
easy_dev_labels = np.load('/content/drive/MyDrive/ARC/Unified QA/easy_dev_labels.npy')

In [ ]:
easy_test_input_ids = np.load('/content/drive/MyDrive/ARC/Unified QA/easy_test_input_ids.npy')
easy_test_attention_mask = np.load('/content/drive/MyDrive/ARC/Unified QA/easy_test_attention_mask.npy')
easy_test_labels = np.load('/content/drive/MyDrive/ARC/Unified QA/easy_test_labels.npy')

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
class arc_dataset(Dataset):

    def __init__(self, input_ids, attention_mask, decoder_input_ids, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.decoder_input_ids = decoder_input_ids
        self.labels = labels
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        sample = {'input_ids': self.input_ids[idx],
                  'attention_mask': self.attention_mask[idx],
                  'decoder_input_ids':self.decoder_input_ids[idx],
                  'labels':self.labels[idx]}
        
        return sample

In [ ]:
train_dataset = arc_dataset(easy_train_input_ids, easy_train_attention_mask, easy_train_decoder_input_ids, easy_train_labels)

In [ ]:
dev_dataset = arc_dataset(easy_dev_input_ids, easy_dev_attention_mask, easy_dev_decoder_input_ids, easy_dev_labels)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)
dev_dataloader = DataLoader(dev_dataset, batch_size=4, shuffle=True, num_workers=4)

In [ ]:
from transformers import T5ForConditionalGeneration, AdamW

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = T5ForConditionalGeneration.from_pretrained("allenai/unifiedqa-t5-base")

model.cuda()

Some weights of the model checkpoint at allenai/unifiedqa-t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

In [ ]:
from transformers import get_linear_schedule_with_warmup

In [ ]:
# This variable contains all of the hyperparemeter information our training loop needs
# Parameters:
lr = 1e-4
max_grad_norm = 1.0
num_training_steps = 1000
num_warmup_steps = 100
warmup_proportion = float(num_warmup_steps) / float(num_training_steps)  # 0.1

### In Transformers, optimizer and schedules are splitted and instantiated like this:
optimizer = AdamW(model.parameters(), lr=lr)  # To reproduce BertAdam specific behavior set correct_bias=False
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)  # PyTorch scheduler

# Store our loss and accuracy for plotting
train_loss_set = []

In [ ]:
# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

In [ ]:
for epoch in range(epochs):
    # Training
    # Set our model to training mode (as opposed to evaluation mode)
    model.train()
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    running_loss = 0

    # Train the data for one epoch
    for step, batch in enumerate(train_dataloader):
        # Add batch to GPU
        # batch = tuple(t.to(device) for t in batch)

        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_decoder_ids, b_labels = batch['input_ids'], batch['attention_mask'], batch['decoder_input_ids'], batch['labels']

        ###############Bug fix code####################
        b_input_ids = b_input_ids.type(torch.LongTensor)
        b_input_mask = b_input_mask.type(torch.LongTensor)
        b_decoder_ids = b_decoder_ids.type(torch.LongTensor)
        b_labels = b_labels.type(torch.LongTensor)

        b_input_ids = b_input_ids.to(device)
        b_input_mask = b_input_mask.to(device)
        b_decoder_ids = b_decoder_ids.to(device)
        b_labels = b_labels.to(device)
         ############################################
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(input_ids = b_input_ids, attention_mask=b_input_mask, decoder_input_ids=b_decoder_ids, labels=b_labels)
        loss, logits = outputs[:2]

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)  # Gradient clipping is not in AdamW anymore (so you can use amp without issue)
        optimizer.step()
        scheduler.step()

        running_loss = loss.item()

        if step%100 == 99:
            print(f'Epoch:{epoch} Batch:{step} Loss:{running_loss}')
            running_loss = 0

print('Training finished.')

Epoch:0 Batch:99 Loss:0.27922549843788147
Epoch:0 Batch:199 Loss:0.07268071174621582
Epoch:0 Batch:299 Loss:0.10768556594848633
Epoch:0 Batch:399 Loss:0.05161546543240547
Epoch:0 Batch:499 Loss:0.13190971314907074
Epoch:1 Batch:99 Loss:0.05984129756689072
Epoch:1 Batch:199 Loss:0.07223325222730637
Epoch:1 Batch:299 Loss:0.003910482861101627
Epoch:1 Batch:399 Loss:0.07332498580217361
Epoch:1 Batch:499 Loss:0.10768122225999832
Epoch:2 Batch:99 Loss:0.00032124866265803576
Epoch:2 Batch:199 Loss:0.052936945110559464
Epoch:2 Batch:299 Loss:0.05737551301717758
Epoch:2 Batch:399 Loss:0.0011466998839750886
Epoch:2 Batch:499 Loss:0.029311392456293106
Epoch:3 Batch:99 Loss:0.047593243420124054
Epoch:3 Batch:199 Loss:0.00819129403680563
Epoch:3 Batch:299 Loss:0.044628120958805084
Epoch:3 Batch:399 Loss:2.61419263551943e-05
Epoch:3 Batch:499 Loss:0.018908299505710602
Training finished.


In [ ]:
loss

tensor(0.0240, device='cuda:0', grad_fn=<NllLossBackward>)

In [ ]:
model.save_pretrained('/content/drive/MyDrive/Unified_QA_2')

### Inference.

In [ ]:
from transformers import T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/Unified_QA_2').to('cuda')

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [ ]:
def run_model(input_ids, **generator_args):
    global tokenizer
    res = model.generate(input_ids, **generator_args)
    return [tokenizer.decode(x) for x in res]

In [ ]:
def predict(array_of_input_ids, options_list):
    '''
    This function takes list of string. Each string is a pair of question and 
    answers separated using \n(all lower cased). 
    For example: 
    which of the following is an example of a physical change? \n (a) lighting a match (b) breaking a glass (c) burning of gasoline (d) rusting of iron

    label_map = {'A':0,
                'B':1,
                'C':2,
                'D':3,
                'E':4,
                '1':0,
                '2':1,
                '3':2,
                '4':3,
                '5':4}

    This function will return predicted option. All answers are converted to 0, 1, 2, 3 and 4. 

    '''
    preds = [] # List to store all predicted labels.
        
    for i in range(len(array_of_input_ids)):

        # Converting input ids to torch tensor of Long type.
        input_ids = torch.from_numpy(array_of_input_ids[i]).type(torch.LongTensor).to('cuda')

        pred = run_model(input_ids.reshape((1,-1)))[0] #Reshaping because model takes input like (batch_size, sequence_length)

        #Pred is in the form like '<pad> answer </s>'. So we have to remove <pad> and </s>.
        pred = pred.replace('<pad>','')
        pred = pred.replace('</s>','')
        pred = pred.lstrip()

        options_list_i = options_list[i]

        label = None
        for j in range(len(options_list_i)):
            if pred in options_list_i[j]:
                label = j
        
        if label == None:
            label = random.randint(0,4)       

        preds.append(label)

    return preds

In [ ]:
easy_train_preds = predict(easy_train_input_ids, df_easy_train['options_list'].values)

In [ ]:
easy_dev_preds = predict(easy_dev_input_ids, df_easy_dev['options_list'].values)

In [ ]:
easy_test_preds = predict(easy_test_input_ids, df_easy_test['options_list'].values)

In [ ]:
a = accuracy_score(easy_train_labels, easy_train_preds)
print(f'Easy train accuracy is {a}')

b = accuracy_score(easy_dev_labels, easy_dev_preds)
print(f'Easy dev accuracy score is {b}')

c = accuracy_score(easy_test_labels, easy_test_preds)
print(f'Easy test accuracy score is {c}')

Easy train accuracy is 0.9098178587294535
Easy dev accuracy score is 0.47368421052631576
Easy test accuracy score is 0.4802188552188552


# Fine tuning Unified QA on ARC with using context.

### Preparing Data.

In [67]:
#Context derived using DRD and ARD models.
df_easy_train = pd.read_csv('/content/drive/MyDrive/arc_train_context')
df_easy_dev = pd.read_csv('/content/drive/MyDrive/arc_dev_context')
df_easy_test = pd.read_csv('/content/drive/MyDrive/arc_test_context')

In [68]:
label_map = {'A':0,
             'B':1,
             'C':2,
             'D':3,
             'E':4,
             '1':0,
             '2':1,
             '3':2,
             '4':3,
             '5':4}

train_labels = df_easy_train['AnswerKey'].map(label_map)
dev_labels = df_easy_dev['AnswerKey'].map(label_map)
test_labels = df_easy_test['AnswerKey'].map(label_map)

In [69]:
def answer(answer_key, options_list):

    label_map = {'A':0,
             'B':1,
             'C':2,
             'D':3,
             'E':4,
             '1':0,
             '2':1,
             '3':2,
             '4':3,
             '5':4}
    
    number = label_map.get(answer_key)
    answer = ast.literal_eval(options_list)[number]
    return answer
    
df_easy_train['Answer'] = df_easy_train[['AnswerKey', 'options_list']].apply(lambda x: answer(*x), axis=1)
df_easy_dev['Answer'] = df_easy_dev[['AnswerKey', 'options_list']].apply(lambda x: answer(*x), axis=1)
df_easy_test['Answer'] = df_easy_test[['AnswerKey', 'options_list']].apply(lambda x: answer(*x), axis=1)

In [70]:
def context_preprocess(context):
    context = re.sub('\s+\W+\s+', ' ', context)
    context = re.sub('\s+',' ', context)

    return context

In [71]:
df_easy_train['context'] = df_easy_train['context'].map(context_preprocess)

In [72]:
df_easy_dev['context'] = df_easy_dev['context'].map(context_preprocess)
df_easy_test['context'] = df_easy_test['context'].map(context_preprocess)

In [21]:
df_easy_train.head()

,AnswerKey,question,only_question,quesiton_length,only_answers,context,options_list,option_1_docs,option_2_docs,option_3_docs,option_4_docs,option_5_docs,Answer
0,B,Which factor will most likely cause a person t...,Which factor will most likely cause a person t...,66,(A) a leg muscle relaxing after exercise (B) a...,-' Which joint of the leg con- tains most musc...,"['a leg muscle relaxing after exercise ', 'bac...","{""-' Which joint of the leg con- tains most mu...",{'When the larvae of Trichinella worms get int...,{'Environmental perturbations are most likely ...,{'Carbohydrates will most likely be the larges...,{'Which technique was most likely used to deve...,bacterial population the bloodstream
1,B,Lichens are symbiotic organisms made of green ...,Lichens are symbiotic organisms made of green ...,139,(A) carbon dioxide (B) food (C) protection (D)...,Lichens are comprised of blue-green or green a...,"['carbon dioxide ', 'food ', 'protection ', 'w...","{""The hydras let these algae live inside them ...","{'algae near the surface of ponds, lakes, and ...","{""In this symbiotic relationship, the algae ga...",{'This is a small genus of marine and brackish...,{'Lichens are comprised of blue-green or green...,food
2,D,When a switch is used in an electrical circuit...,When a switch is used in an electrical circuit...,63,(A) cause the charge to build. (B) increase an...,When the circuit switch is open (Figure 2) cur...,"['cause the charge to build. ', 'increase and ...",{'If an ordinary switch is used to control the...,{'And when the electrical switch was turned OF...,{'If an ordinary switch is used to control the...,{'When the circuit switch is open (Figure 2) c...,{'If an ordinary switch is used to control the...,stop start flow of current.
3,A,Which of the following is an example of an ass...,Which of the following is an example of an ass...,61,(A) contact lens (B) motorcycle (C) raincoat (...,Which of the following is an example of an ang...,"['contact lens ', 'motorcycle ', 'raincoat ', ...",{'An instrument for measuring the degree of cu...,{'The characters are typically members of an o...,"{'[DUSSEL hurriedly takes off the raincoat, gi...",{'coffee\tn\ta beverage consisting of an infus...,{'Which of the following is an example of an a...,contact lens
4,3,"Rocks are classified as igneous, metamorphic, ...","Rocks are classified as igneous, metamorphic, ...",74,(1) their color (2) their shape (3) how they f...,"- formed by “changing” preexisting igneous, se...","['their color ', 'shape ', 'how they formed ',...","{'In one simplistic and linear example, elemen...",{'Western carrot cultivars are commonly classi...,"{'- formed by “changing” preexisting igneous, ...",{'This group of minerals is of common occur- r...,"{'The second class of rocks, sedimentary,\xa0 ...",how they formed


In [10]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained('t5-small')

In [11]:
MAX_LEN = 512
decoder_max_len = 30

In [22]:
def arc_preprocessor(dataset, tokenizer):
    '''
    This function will convert a given context, question, choices in a format:

    input: question \n options \n context </s>
    target: label </s>

    After converting in this format the data will be tokenized using a given tokenizer.
    This function will return 4 arrays namely, input_ids, attention_mask, token_type_ids and labels.
    '''
    global MAX_LEN
    all_input_ids = []
    all_attention_mask = []
    all_decoder_input_ids = []
    all_labels = []

    for i in range(len(dataset)):
        
        context = ' '.join(dataset['context'].iloc[i].split()[:350]) #Limiting the maximum length of context to 350 words.
        question = dataset['only_question'].iloc[i]
        options = dataset['only_answers'].iloc[i]
        target = dataset['Answer'].iloc[i]
        choice_features = []

        input_string = question + ' ' + '\\n' + ' ' + options + ' ' + '\\n' + ' ' + context + ' ' + '</s>'
        decoder_input = tokenizer.pad_token + ' ' + target
        target = target + ' ' + '</s>'

        input_ids = tokenizer.encode(input_string, truncation=True, max_length=MAX_LEN)
        decoder_input_ids = tokenizer.encode(decoder_input, max_length=decoder_max_len, truncation=True) #Max length of a answer is 23
        labels = tokenizer.encode(target, max_length=decoder_max_len, truncation=True)

        attention_mask = [1] * len(input_ids)

        padding_id = tokenizer.pad_token_id
        padding_length = MAX_LEN - len(input_ids)

        input_ids = input_ids + [padding_id]*padding_length
        attention_mask = attention_mask + [0]*padding_length

        deocder_padding_length = decoder_max_len - len(decoder_input_ids)

        decoder_input_ids = decoder_input_ids + [padding_id]*deocder_padding_length
        labels = labels + [padding_id]*deocder_padding_length

        assert len(input_ids) == MAX_LEN
        assert len(attention_mask) == MAX_LEN

        all_input_ids.append(np.asarray(input_ids, dtype='int32'))
        all_attention_mask.append(np.asarray(attention_mask, dtype='int32'))
        all_decoder_input_ids.append(np.asarray(decoder_input_ids, dtype='int32'))
        all_labels.append(np.asarray(labels, dtype='int32'))

    return all_input_ids, all_attention_mask, all_decoder_input_ids, all_labels

In [23]:
easy_train_input_ids, easy_train_attention_mask, easy_train_decoder_input_ids, easy_train_labels = arc_preprocessor(df_easy_train, tokenizer)

In [24]:
easy_dev_input_ids, easy_dev_attention_mask, easy_dev_decoder_input_ids, easy_dev_labels = arc_preprocessor(df_easy_dev, tokenizer)

In [25]:
easy_test_input_ids, easy_test_attention_mask, _, easy_test_labels = arc_preprocessor(df_easy_test, tokenizer)

In [26]:
easy_train_input_ids = np.asarray(easy_train_input_ids, dtype='int32')
easy_train_attention_mask = np.asarray(easy_train_attention_mask, dtype='int32')
easy_train_decoder_input_ids = np.asarray(easy_train_decoder_input_ids, dtype='int32')
easy_train_labels = np.asarray(easy_train_labels, dtype='int32')

In [27]:
easy_dev_input_ids = np.asarray(easy_dev_input_ids, dtype='int32')
easy_dev_attention_mask = np.asarray(easy_dev_attention_mask, dtype='int32')
easy_dev_decoder_input_ids = np.asarray(easy_dev_decoder_input_ids, dtype='int32')
easy_dev_labels = np.asarray(easy_dev_labels, dtype='int32')

In [28]:
easy_test_input_ids = np.asarray(easy_test_input_ids, dtype='int32')
easy_test_attention_mask = np.asarray(easy_test_attention_mask, dtype='int32')
easy_test_labels = np.asarray(easy_test_labels, dtype='int32')

In [46]:
np.save('/content/drive/MyDrive/ARC/Unified QA with context/easy_train_input_ids', easy_train_input_ids)
np.save('/content/drive/MyDrive/ARC/Unified QA with context/easy_train_attention_mask', easy_train_attention_mask)
np.save('/content/drive/MyDrive/ARC/Unified QA with context/easy_train_decoder_input_ids', easy_train_decoder_input_ids)
np.save('/content/drive/MyDrive/ARC/Unified QA with context/easy_train_labels', easy_train_labels)

In [47]:
np.save('/content/drive/MyDrive/ARC/Unified QA/easy_dev_input_ids', easy_dev_input_ids)
np.save('/content/drive/MyDrive/ARC/Unified QA/easy_dev_attention_mask', easy_dev_attention_mask)
np.save('/content/drive/MyDrive/ARC/Unified QA/easy_dev_decoder_input_ids', easy_dev_decoder_input_ids)
np.save('/content/drive/MyDrive/ARC/Unified QA/easy_dev_labels', easy_dev_labels)

In [48]:
np.save('/content/drive/MyDrive/ARC/Unified QA/easy_test_input_ids', easy_test_input_ids)
np.save('/content/drive/MyDrive/ARC/Unified QA/easy_test_attention_mask', easy_test_attention_mask)
np.save('/content/drive/MyDrive/ARC/Unified QA/easy_test_labels', easy_test_labels)

### Training.

In [9]:
easy_train_input_ids = np.load('/content/drive/MyDrive/ARC/Unified QA with context/easy_train_input_ids.npy')
easy_train_attention_mask = np.load('/content/drive/MyDrive/ARC/Unified QA with context/easy_train_attention_mask.npy')
easy_train_decoder_input_ids = np.load('/content/drive/MyDrive/ARC/Unified QA with context/easy_train_decoder_input_ids.npy')
easy_train_labels = np.load('/content/drive/MyDrive/ARC/Unified QA with context/easy_train_labels.npy')

In [10]:
easy_dev_input_ids = np.load('/content/drive/MyDrive/ARC/Unified QA/easy_dev_input_ids.npy')
easy_dev_attention_mask = np.load('/content/drive/MyDrive/ARC/Unified QA/easy_dev_attention_mask.npy')
easy_dev_decoder_input_ids = np.load('/content/drive/MyDrive/ARC/Unified QA/easy_dev_decoder_input_ids.npy')
easy_dev_labels = np.load('/content/drive/MyDrive/ARC/Unified QA/easy_dev_labels.npy')

In [11]:
easy_test_input_ids = np.load('/content/drive/MyDrive/ARC/Unified QA/easy_test_input_ids.npy')
easy_test_attention_mask = np.load('/content/drive/MyDrive/ARC/Unified QA/easy_test_attention_mask.npy')
easy_test_labels = np.load('/content/drive/MyDrive/ARC/Unified QA/easy_test_labels.npy')

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader

In [11]:
class arc_dataset(Dataset):

    def __init__(self, input_ids, attention_mask, decoder_input_ids, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.decoder_input_ids = decoder_input_ids
        self.labels = labels
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        sample = {'input_ids': self.input_ids[idx],
                  'attention_mask': self.attention_mask[idx],
                  'decoder_input_ids':self.decoder_input_ids[idx],
                  'labels':self.labels[idx]}
        
        return sample

In [12]:
train_dataset = arc_dataset(easy_train_input_ids, easy_train_attention_mask, easy_train_decoder_input_ids, easy_train_labels)

In [13]:
dev_dataset = arc_dataset(easy_dev_input_ids, easy_dev_attention_mask, easy_dev_decoder_input_ids, easy_dev_labels)

In [14]:
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)
dev_dataloader = DataLoader(dev_dataset, batch_size=4, shuffle=True, num_workers=4)

In [15]:
from transformers import T5ForConditionalGeneration, AdamW

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = T5ForConditionalGeneration.from_pretrained("allenai/unifiedqa-t5-base")

model.cuda()

Some weights of the model checkpoint at allenai/unifiedqa-t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

In [17]:
from transformers import get_linear_schedule_with_warmup

In [20]:
len(train_dataloader) * 4

2252

In [21]:
# This variable contains all of the hyperparemeter information our training loop needs
# Parameters:
lr = 1e-4
max_grad_norm = 1.0
num_training_steps = 2252
num_warmup_steps = 225
warmup_proportion = float(num_warmup_steps) / float(num_training_steps)  # 0.1

### In Transformers, optimizer and schedules are splitted and instantiated like this:
optimizer = AdamW(model.parameters(), lr=lr)  # To reproduce BertAdam specific behavior set correct_bias=False
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)  # PyTorch scheduler

# Store our loss and accuracy for plotting
train_loss_set = []

In [22]:
# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

In [23]:
for epoch in range(epochs):
    # Training
    # Set our model to training mode (as opposed to evaluation mode)
    model.train()
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    running_loss = 0

    # Train the data for one epoch
    for step, batch in enumerate(train_dataloader):
        # Add batch to GPU
        # batch = tuple(t.to(device) for t in batch)

        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_decoder_ids, b_labels = batch['input_ids'], batch['attention_mask'], batch['decoder_input_ids'], batch['labels']

        ###############Bug fix code####################
        b_input_ids = b_input_ids.type(torch.LongTensor)
        b_input_mask = b_input_mask.type(torch.LongTensor)
        b_decoder_ids = b_decoder_ids.type(torch.LongTensor)
        b_labels = b_labels.type(torch.LongTensor)

        b_input_ids = b_input_ids.to(device)
        b_input_mask = b_input_mask.to(device)
        b_decoder_ids = b_decoder_ids.to(device)
        b_labels = b_labels.to(device)
         ############################################
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(input_ids = b_input_ids, attention_mask=b_input_mask, decoder_input_ids=b_decoder_ids, labels=b_labels)
        loss, logits = outputs[:2]

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)  # Gradient clipping is not in AdamW anymore (so you can use amp without issue)
        optimizer.step()
        scheduler.step()

        running_loss = loss.item()

        if step%100 == 99:
            print(f'Epoch:{epoch} Batch:{step} Loss:{running_loss}')
            running_loss = 0

print('Training finished.')

Epoch:0 Batch:99 Loss:0.4524567425251007
Epoch:0 Batch:199 Loss:0.09563900530338287
Epoch:0 Batch:299 Loss:0.017218150198459625
Epoch:0 Batch:399 Loss:0.016093354672193527
Epoch:0 Batch:499 Loss:0.028746332973241806
Epoch:1 Batch:99 Loss:0.01116793043911457
Epoch:1 Batch:199 Loss:0.07901991903781891
Epoch:1 Batch:299 Loss:0.05968160182237625
Epoch:1 Batch:399 Loss:0.00817718543112278
Epoch:1 Batch:499 Loss:0.02017420344054699
Epoch:2 Batch:99 Loss:0.0716371014714241
Epoch:2 Batch:199 Loss:0.021269764751195908
Epoch:2 Batch:299 Loss:0.010799265466630459
Epoch:2 Batch:399 Loss:0.02806326188147068
Epoch:2 Batch:499 Loss:0.003330040955916047
Epoch:3 Batch:99 Loss:0.07855100929737091
Epoch:3 Batch:199 Loss:0.0003087057557422668
Epoch:3 Batch:299 Loss:0.022966571152210236
Epoch:3 Batch:399 Loss:0.0022621345706284046
Epoch:3 Batch:499 Loss:0.00019682505808304995
Training finished.


In [24]:
loss

tensor(0.0002, device='cuda:0', grad_fn=<NllLossBackward>)

In [25]:
model.save_pretrained('/content/drive/MyDrive/Unified_QA_with_context/')

### Inference.

In [12]:
from transformers import T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/Unified_QA_with_context').to('cuda')

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [14]:
def run_model(input_ids, **generator_args):
    global tokenizer
    res = model.generate(input_ids, **generator_args)
    return [tokenizer.decode(x) for x in res]

In [32]:
def predict(array_of_input_ids, options_list):
    '''
    This function takes list of string. Each string is a pair of question and 
    answers separated using \n(all lower cased). 
    For example: 
    which of the following is an example of a physical change? \n (a) lighting a match (b) breaking a glass (c) burning of gasoline (d) rusting of iron

    label_map = {'A':0,
                'B':1,
                'C':2,
                'D':3,
                'E':4,
                '1':0,
                '2':1,
                '3':2,
                '4':3,
                '5':4}

    This function will return predicted option. All answers are converted to 0, 1, 2, 3 and 4. 

    '''
    preds = [] # List to store all predicted labels.
        
    for i in range(len(array_of_input_ids)):

        # Converting input ids to torch tensor of Long type.
        input_ids = torch.from_numpy(array_of_input_ids[i]).type(torch.LongTensor).to('cuda')

        pred = run_model(input_ids.reshape((1,-1)))[0] #Reshaping because model takes input like (batch_size, sequence_length)

        #Pred is in the form like '<pad> answer </s>'. So we have to remove <pad> and </s>.
        pred = pred.replace('<pad>','')
        pred = pred.replace('</s>','')
        pred = pred.lstrip()

        options_list_i = ast.literal_eval(options_list[i])

        label = None
        for j in range(len(options_list_i)):
            if pred in options_list_i[j]:
                label = j
        
        if label == None:
            label = random.randint(0,4)       

        preds.append(label)

    return preds

In [33]:
easy_train_preds = predict(easy_train_input_ids, df_easy_train['options_list'])

In [34]:
easy_dev_preds = predict(easy_dev_input_ids, df_easy_dev['options_list'])

In [35]:
easy_test_preds = predict(easy_test_input_ids, df_easy_test['options_list'])

In [42]:
a = accuracy_score(easy_train_labels, easy_train_preds)
print(f'Easy train accuracy is {a}')

b = accuracy_score(easy_dev_labels, easy_dev_preds)
print(f'Easy dev accuracy score is {b}')

c = accuracy_score(easy_test_labels, easy_test_preds)
print(f'Easy test accuracy score is {c}')

Easy train accuracy is 0.9680142159040427
Easy dev accuracy score is 0.543859649122807
Easy test accuracy score is 0.5420875420875421


# Reference Links:

1. https://github.com/sobamchan/race-bert/blob/master/race.py